<h1 align="center">Utilizing Markov Chains with the N-Gram Method to Suggest New Sentences from a Base Text</h1>

<br>

<b>Project:</b> Markov Chain and N-Grams  
<b>Class:</b> Cpts 315 Washington State University  
<b>Description:</b> Final Project  
<b>By:</b> Kyle Hurd

# Introduction - Markov Chain

A <b>Markov Chain</b> is a model that makes predictions based on a sequence of potential states. It will weigh the probability in which a set of states will be in the sequence and use this information to generate a new sequence. The defining characteristic in which probability is weighed is exclusively dependent on a current state and a passage of time. In other words, past states do not influence the Markov Chain, only the current state. The transition from the current state to the next state in a <b>Markov Chain</b> is determined by using probabilty. The algorithm will consider the probability of a current state transitioning to a potential state and transition based on frequency. 

---

Here is an example to explain the behavior described above. Suppose we have a state machine consisting of two states: State <b>q0</b> represents our initial state. Anything traveling to this state will produce a binary value of <b>1</b>. State <b>q1</b> represents the second state which will produce binary <b>0</b>.

<p align="center">


<img src="imgs/two_state_machine.png"/>


</p>

Let us assume the probability in which state <b>q0</b> will transition to <b>q1</b> is 50/50, vice versa.<br><br> 

```
    P(q0|q0) = 0.50
    P(q0|q1) = 0.50
    P(q1|q0) = 0.50
    P(q1|q1) = 0.50
```

The above probabilities can be read as follows:  

```
For P(q0|q0), this describes the probability that a transition from state q0 -> q0 will have a frequency of 50%.  

For P(q0|q1), this describes the probability that a transition from state q0 -> q1 will have a frequency of 50%.  

For P(q1|q0), this describes the probability that a transition from state q1 -> q0 will have a frequency of 50%.  

For P(q1|q1), this describes the probability that a transition from state q1 -> q1 will have a frequency of 50%.
```

---
In the next example, we will generate a generic state machine with a total of three states, increasing the number of potential transitions to three.

<p align="center">


<img src="imgs/three_state_machine.png"/>


</p>

In this example, we only consider the probabilty of transition from one state to another: information such as the alphabet and grammar are ignored.  

The probabilities are listed below:

```
P(q0|q0) = 0.20
P(q0|q1) = 0.40
P(q0|q2) = 0.20

P(q1|q0) = 0.50
P(q1|q1) = 0.25
P(q1|q1) = 0.25

P(q2|q0) = 0.10
P(q2|q1) = 0.80
P(q2|q2) = 0.10
```

## Setting up the Code

---

First we need to initalize a class with all the information we will need
for a Markoc Chain. Here are a few that we will need:

- a list of words from our source text for which to build the chain.
- a dictionary to store the n-grams and list of next words.
- the name of the source file (in case of accessing later.

I also thought it would be cool to hold some information regarding the total number of
characters, words, and unique words in the text. We will store this information in a dataclass.


In [1]:
# JDC used to extend a Class Object in Jupyter Notebook
import jdc


import random
from colorama import Fore, Style
from dataclasses import dataclass
from functools import reduce

HUNGER_GAMES_FILENAME = './data/hunger_games.txt'

STOP_CHARACTERS = '.?!'
STOP_WORDS = ['Dr.', 'Jr.', 'Sr.', 'Mr.', 'Mrs.', 'Ms.', 'Miss.', 'Prof.']
FULL_QUOTE = '"'

## TextSpecs DataClass

In [2]:
@dataclass
class TextSpecs:
    num_chars: int = 0
    num_words: int = 0
    num_unique_words: int = 0
        
        
    def _populate(self, num_chars: int, num_words: int, num_unique_words: int):
        self.num_chars += num_chars
        self.num_words += num_words
        self.num_unique_words += num_unique_words
        
        
    def display_specs(self):
        print(f'{Style.BRIGHT}{Fore.LIGHTGREEN_EX}{"#" * 18}' \
              f'{"#" * (len(str(self.num_unique_words)) + 1)}{Style.RESET_ALL}')
        
        print(f'{Style.BRIGHT}num chars: {Style.RESET_ALL}{self.num_chars}{Style.RESET_ALL} ')
        print(f'{Style.BRIGHT}num words: {Style.RESET_ALL}{self.num_words}{Style.RESET_ALL} ')
        print(f'{Style.BRIGHT}num unique words: {Style.RESET_ALL}{self.num_unique_words}{Style.RESET_ALL} ')
        
        print(f'{Style.BRIGHT}{Fore.LIGHTGREEN_EX}{"#" * 18}' \
              f'{"#" * (len(str(self.num_unique_words)) + 1)}{Style.RESET_ALL}')

## MarkovChain Class

---

Below is the initializer for the MarkovChain. It also defines `display_specs` which utilizes the `TextSpecs`
dataclass defined above to print out information regarding the text(s) we are using. It is important to note
that `TextSpecs.populate()` keeps the original values and adds to it using the augmented assignment operator. This
means we should not call these functions directly in practice, but should use the wrapper function defined further
down the page.

In [3]:
class MarkovChain:
    
    def __init__(self, filenames: list, N: int=3, stop_characters=None, stop_words=None):
        self.initial_words = []
        self.n_grams = {}
        self.starting_n_grams = []
        self.filenames = filenames
        self.stop_characters = stop_characters
        self.stop_words = stop_words
        self.N = N
        self.specs = TextSpecs()
        
    
    def display_specs(self):
        print(f'{Style.BRIGHT}Files:{Style.RESET_ALL}')
        for filename in self.filenames:
            print(f'{Style.BRIGHT}{Fore.LIGHTRED_EX}-{Style.RESET_ALL} {filename}')
        self.specs.display_specs()


    For the methods below, these will be wrapped with a function to keep the proper
    states of the initialized variables within `MarkovChain` and `TextSpecs`

## MarkovChain._init_words()

In [4]:
%%add_to MarkovChain

def _init_words(self):
    
    for filename in self.filenames:
        with open(filename, 'r') as f:
            chars = f.read()
            words = chars.split()
            unique_words = set(words)
            self.specs._populate(len(chars), len(words), len(unique_words))
            self.initial_words.extend(words)

In [5]:
mc = MarkovChain(filenames=[HUNGER_GAMES_FILENAME],
                 N=3,
                 stop_characters=STOP_CHARACTERS,
                 stop_words=STOP_WORDS
                )

mc._init_words()
mc.display_specs()

print(f'\n{Style.BRIGHT}Preview of the Text:{Style.RESET_ALL}')
for i in range(50):
    print(mc.initial_words[i], end=' ')

Files:
- ./data/hunger_games.txt
#######################
num chars: 27113 
num words: 5081 
num unique words: 1914 
#######################

Preview of the Text:
When I wake up, the other side of the bed is cold. My fingers stretch out, seeking Prim’s warmth but finding only the rough canvas cover of the mattress. She must have had bad dreams and climbed in with our mother. Of course, she did. This is the day of 

## MarkovChain._create_ngram_dict()

This is where the probability between states comes in to play. Note here, when we add the next word beyond the
n-gram (the Nth + 1 word), we allow duplicates into the list. This means we could recieve a list such as 
`[the, The, the, tiny]` where 75% of the words are `the` and 25% are `tiny`. When selecting from this list
in the future, this means that if we select from the bag of words randomly, we should see a selection of the
word `the` approximately 75% of the time.

In [6]:
%%add_to MarkovChain

def _create_ngram_dict(self):
    n_grams = zip(*[self.initial_words[i:] for i in range(self.N + 1)])
    for n_gram in n_grams:
        key = n_gram[:self.N]
        next_word = n_gram[-1]
        self.n_grams[key] = self.n_grams.get(key, []) + [next_word]
        
        
def _create_starting_ngram_list(self):
    
    is_valid      = lambda g: g[0] not in self.stop_words and (g[1][0].isupper() or g[1][0] in ["'", '"'])
    in_stop_chars = lambda g: g[0][-1] in self.stop_characters
    
    n_grams = zip(*[self.initial_words[i:] for i in range(self.N + 1)])
    for n_gram in n_grams:
        if in_stop_chars(n_gram) and is_valid(n_gram):
            self.starting_n_grams.append(n_gram[1:])
        

## Preview Results from the N-Grams and the Entries

In [7]:
mc.n_grams = {} # Only used because we are calling this multiple times.

mc._create_ngram_dict()
n_gram_vals = list(mc.n_grams.values())

print(f'\n{Style.BRIGHT}Preview of N-Grams:{Style.RESET_ALL}')
for i, key in enumerate(mc.n_grams.keys()):
    if i == 5:
        break
    print(f'{Style.BRIGHT}- {Style.RESET_ALL}{Fore.LIGHTGREEN_EX}{key}{Style.RESET_ALL}')


Preview of N-Grams:
- ('When', 'I', 'wake')
- ('I', 'wake', 'up,')
- ('wake', 'up,', 'the')
- ('up,', 'the', 'other')
- ('the', 'other', 'side')


In [8]:
print(f'\n{Style.BRIGHT}Preview of the N-Gram Entries:{Style.RESET_ALL}')
for n_gram in n_gram_vals[:5]:
    print(f'{Style.BRIGHT}- {Style.RESET_ALL}', end='')
    for gram in n_gram:
        print(f'{Fore.LIGHTGREEN_EX}{gram}{Style.RESET_ALL}', end=' ')
    print()
    
n_gram_new = list(filter(lambda x: len(x) > 1, n_gram_vals))
for entries in n_gram_new[:5]:
    print(f'{Style.BRIGHT}- {Style.RESET_ALL}', end='')
    for entry in entries:
        print(f'{Fore.LIGHTGREEN_EX}{entry}{Style.RESET_ALL}', end=' ')
    print()


Preview of the N-Gram Entries:
- up, 
- the 
- other 
- side 
- of 
- had really 
- day closest 
- 12, 12. 
- nicknamed is 
- only try 


In [9]:
mc.starting_n_grams = [] # Only used because we are calling this multiple times.

mc._create_starting_ngram_list()
print(f'\n{Style.BRIGHT}Preview of the N-Gram Starters:{Style.RESET_ALL}')
for n_gram in mc.starting_n_grams[:10]:
    print(f'{Style.BRIGHT}- {Style.RESET_ALL}{Fore.LIGHTGREEN_EX}{n_gram}{Style.RESET_ALL}')


Preview of the N-Gram Starters:
- ('My', 'fingers', 'stretch')
- ('She', 'must', 'have')
- ('Of', 'course,', 'she')
- ('This', 'is', 'the')
- ('I', 'prop', 'myself')
- ('There’s', 'enough', 'light')
- ('My', 'little', 'sister,')
- ('In', 'sleep,', 'my')
- ('Prim’s', 'face', 'is')
- ('My', 'mother', 'was')


## MarkovChain.generate_sentence [1st iteration]

This is the initial draft of the generate_sentence method. It it actually generates
understandable text and was my first solution that showed a promising output return.
The key to making this function work was to separate the n-grams with n-grams that can
start a sentence. These are `self.n_grams` and `self.starting_n_grams`, respectively.

By intializing with an n-gram that is the beginning of a sentence,
We can start the chain from the beginning of the sentence instead of midway through or at the end.
This solution shows another issue: the sentences that it has generated, although somewhat coherent,
end midway through a sentence.

In [10]:
%%add_to MarkovChain

def generate_sentence(self):
    
    length_sentence = random.randint(4, 15)  
    seed = random.choice(self.starting_n_grams)
    output = [x for x in seed]
    for _ in range(length_sentence):
        word = random.choice(self.n_grams[seed])
        seed = tuple(list(seed[1:]) + [word])
        output.append(word)
        
    return output

In [11]:
print(f'{Style.BRIGHT}Preview of Generated Sentences:{Style.RESET_ALL}')
for _ in range(5):
    print(f'{Style.BRIGHT}{Fore.RED}- {Style.RESET_ALL}', end='')
    for word in mc.generate_sentence():
        print(word, end=' ')
    print()

Preview of Generated Sentences:
- Most of the Peacekeepers turn a blind eye to the few of us 
- Even so, I always take a moment to listen carefully for the 
- We rarely talk, which suits us both just fine. Today her 
- Gale, who is eighteen and has been either helping or 
- My mother wears a fine dress from her apothecary days. Prim is in my first 


## MarkovChain.generate_sentence [2nd iteration]

In this second iteration, I address the issue where the sentence ends halfway through. Additionally,
a few times in the iteration one implementation, there is sometimes a random quotation that tries to
start a quote or end a quote. This will also be addressed in this iteration. Testing for a quote at
the beginning or end is also quite a difficult problem to fix in a simple way. Somehow, we have to keep
track of the current state the generator is in (does it need to search for an ending quote, has it seen
a closing quote but no opening quote?). The second problem is more challenging. It is pretty straightforward
to search for a closing quote after seeing an opening, but what are we to do if we see a closing quote?


To be honest, I don't have a good solution to this issue. One "solution" would be to eliminate the quotes
all-together from the generator, but that is no fun. The next solution could be to insert the starting quote
at the start of the previous sentence, but there are too many conditions to consider. For example,

- She said, "Hello, foo! How is bar?"
- "Hello, foo!" she said, "How is bar?"
- "Hello, foo! How is bar?" She said.

In the first example, we can't just insert the start quote at the beginning of the sentence, as that would
be incorrect. Additionally, the second condition is even harder, for we have to potentially insert two quotes
in one sentence! The last example would be the only time where the "fix" would work as intended. The problem
with the first two examples are the word "said" or "she" can be replaced with too many different words such
as "He", "Jared", or "exlaimed", "cried."

In [12]:
%%add_to MarkovChain

def generate_sentence(self):
    
    length_sentence = random.randint(4, 15)  
    seed = random.choice(self.starting_n_grams)
    
    
    output = [x for x in seed]
    is_quote = reduce(lambda base, word: (word[0] == FULL_QUOTE) or base, output, False)

    for _ in range(length_sentence):
        word, seed = self._generate_word(seed, is_quote)
        output.append(word)
        
    self._end_sentence(output, seed, is_quote)
    return output      


def _generate_word(self, seed, is_quote=False):
    
    not_ending_quote = lambda word: word[-1] != FULL_QUOTE
    
    words = self.n_grams[seed]
    words = [word for word in words if not_ending_quote(word)] if not is_quote else words
    word = random.choice(words)
    seed = tuple(list(seed[1:]) + [word])
    
    return word, seed
    

def _end_sentence(self, output, seed, is_quote=False):
    
    in_stop_characters = lambda word: word[-1] in self.stop_characters
    in_stop_words      = lambda word: word in self.stop_words
    
    while not (end := [word for word in self.n_grams[seed] if in_stop_characters(word) and not in_stop_words(word)]):
        word, seed = self._generate_word(seed, is_quote)
        is_quote |= word[0] == FULL_QUOTE  # if quote at beginning, make true.
        is_quote &= word[-1] != FULL_QUOTE # if quote at end, make false.
        output.append(word)
        
    word = random.choice(end)
    n_gram = tuple(list(seed[1:]) + [word])
    output.append(word)

In [13]:
print(f'{Style.BRIGHT}Preview of Generated Sentences [Iteration 2]:{Style.RESET_ALL}')
for _ in range(5):
    sentence = mc.generate_sentence()
    print(f'{Style.BRIGHT}{Fore.RED}- {Style.RESET_ALL}', end='')
    for word in sentence:
        print(word, end=' ')
    print()

Preview of Generated Sentences [Iteration 2]:
- But today, despite the bright banners hanging on the buildings, there’s an air of grimness. 
- How could I leave Prim, who is the only person in the world I’m certain I love? 
- Supple leather that has molded to my feet. I pull on trousers, a shirt, tuck my long dark braid up into a cap, and grab my forage bag. 
- I had six when I was just twelve years old.” “That’s not her fault,” I say. 
- Even here, even in the middle of nowhere, you worry someone might overhear you. 
